# Ejercicios Vivo Clase 1


## Setup
Instalo pyspark


In [ ]:
!pip3 install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


##Ejecuta las siguientes celdas para acceder al archivo que usaremos en el ejercicio e instalar spark en esta instancia de colab.

In [ ]:
import requests
with open("./energias.csv", "wb") as f:
    f.write(requests.get("https://raw.githubusercontent.com/engcarlosperezmolero/resources_and_tools/main/data/csv/energias-alternativas.csv").content)

## Crear un RDD y contar el número de registros
Crea un RDD a partir del archivo "./energias.csv" y cuenta el número de registros presentes en él.


In [ ]:
from pyspark import SparkContext, SparkConf

# Configura el SparkContext
conf = SparkConf().setMaster("local").setAppName("Ejercicio1")

# Obtiene el SparkContext existente o crea uno nuevo
sc = SparkContext.getOrCreate(conf)

data = sc.textFile("./energias.csv")
print(type(data))
print(data.count())

<class 'pyspark.rdd.RDD'>
5312


##Posibles errores:


*   ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Ejercicio1, master=local) created by __init__ at <ipython-input-3-61a570d717c2>:3 --> Ya creaste un SparkContexts


*   No encuentra el archivo csv



## Obtener el mínimo, máximo y promedio de la columna "potencia_instalada"
Crea un RDD a partir del archivo "./energias.csv" y obtén el mínimo, máximo y promedio de la columna "potencia_instalada".


In [ ]:
def obtener_potencia(line):
    campos = line.split(",")
    try:
        potencia = float(campos[13])
        return potencia
    except ValueError:
        return None

# Aplico la funcion 'obtener_potencia' a cada linea del RDD
potencias = data.map(obtener_potencia)

# Filtro los valores no nulos
potencias = potencias.filter(lambda x: x is not None)

# Calculo el minimo, el maximo y el promedio
potencia_min = potencias.min()
potencia_max = potencias.max()
potencia_promedio = potencias.mean()

# Imprimo los resultados
print(f"Min: {potencia_min}, Max: {potencia_max}, Promedio: {potencia_promedio}")

Min: 0.0, Max: 752501.256, Promedio: 20086.47435322918


## Filtrar y ordenar registros por "potencia_instalada"
Crea un RDD a partir del archivo "./energias.csv", filtra los registros que tengan "potencia_instalada" mayor a 20 y ordénalos de manera descendente.


In [ ]:
def filtro_potencia_mayor_20(linea):
    campos = linea.split(",")
    try:
        return float(campos[13]) > 20
    except ValueError:
        return False

data_filtrada = data.filter(filtro_potencia_mayor_20)

def obtener_potencia_registro(linea):
    campos = linea.split(",")
    return (float(campos[13]), linea)

data_filtrada_potencia = data_filtrada.map(obtener_potencia_registro)
data_filtrada_potencia_ordenada = data_filtrada_potencia.sortByKey(ascending=False)
registros_ordenados = data_filtrada_potencia_ordenada.map(lambda x: x[1])

#print(registros_ordenados.take(5))
for i in registros_ordenados.take(5):
  print(i)

25,Energias alternativas,445,Energia nuclear,Generacion,MWh,CAMMESA,Mensual,Nacional,PROVINCIA,6,BUENOS AIRES,2018-05-01,752501.256
25,Energias alternativas,445,Energia nuclear,Generacion,MWh,CAMMESA,Mensual,Nacional,PROVINCIA,6,BUENOS AIRES,2016-05-01,749102.828
25,Energias alternativas,445,Energia nuclear,Generacion,MWh,CAMMESA,Mensual,Nacional,PROVINCIA,6,BUENOS AIRES,2018-06-01,741414.599
25,Energias alternativas,445,Energia nuclear,Generacion,MWh,CAMMESA,Mensual,Nacional,PROVINCIA,6,BUENOS AIRES,2016-10-01,740803.27
25,Energias alternativas,445,Energia nuclear,Generacion,MWh,CAMMESA,Mensual,Nacional,PROVINCIA,6,BUENOS AIRES,2018-07-01,706410.365
